# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 марта 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 27 марта, -4 балла после 06:00 3 апреля, -6 баллов после 06:00 10 апреля

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (2 баллов)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы на fit сравнимой со sklearn wine и Speed Dating Data. 
Для этого используем numpy. 

###### Задание 3 (2 балла)
Продемонстрируйте умение работать с Pipeline на данных Speed Dating Data и DecisionTreeClassifier. Нужно в pipeline произвести все необходимые преобразования данных и в конце обучить модель. Задание реализуйте под пунктом Задание 3 (уже написано ниже)

###### Задание 4 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 5 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [2]:
# %%pycodestyle
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
            self.G_supple_function = self.__supple_gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
            self.G_supple_function = self.__supple_entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
            self.G_supple_function = self.__supple_misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __supple_gini(self, cl, s):
        return (1-((cl/s)**2).sum(axis=1)).reshape(-1, 1)

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return l_s * self.__supple_gini(l_c, l_s) +\
            r_s * self.__supple_gini(r_c, r_s)

    def __supple_entropy(self, cl, s):
        return (((cl/s)*(cl/s)**2).sum(axis=1)).reshape(-1, 1)

    def __entropy(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return l_s * self.__supple_entropy(l_c, l_s) +\
            r_s * self.__supple_entropy(r_c, r_s)

    def __supple_misclass(self, cl, s):
        return (1-(cl/s).max(axis=1)).reshape(-1, 1)

    def __misclass(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return l_s * self.__supple_misclass(l_c, l_s) +\
            r_s * self.__supple_misclass(l_c, l_s)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.sqrt(n_feature))]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(n_feature))]

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Сортируем массив с признаками, получаем массив с классами.
        # Получаем колво классов в выборке.
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = y.max()+1
        # Что делает этот блок кода?
        # Берем массив индексов, отсекаем справа и слева куски,
        # по которым точно не пройдет граница.
        # И строим массив правых индексов сменяющихся значений.
        # Поскольку правая граница - прибавляем еще 1.
        splitted_sorted_y = \
            sorted_y[self.min_samples_split:-self.min_samples_split]
        r_border_ids = np.where(splitted_sorted_y[:-1] !=
                                splitted_sorted_y[1:])[0] +\
            (self.min_samples_split + 1)
        if len(r_border_ids) == 0:
            return float('+inf'), None

        # Что делает этот блок кода?
        # Формируем массив one_hot_code - каждая строка отвечает
        # за границу, по которой потенциально может пройти разбиение.
        # Столбцы соответствуют классам. В столбце поднята единица,
        # если разбиение прошло по данному классу.
        # В class_increments получаем сколько элементов было в группах
        # включая первые min_samples_split элементов.
        eq_el_count = r_border_ids - \
            np.append([self.min_samples_split], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + \
            np.bincount(sorted_y[:self.min_samples_split],
                        minlength=class_number)

        # Что делает этот блок кода?
        # Накапливаем в каждой строке элементы каких классов
        # хранятся до каждой границы разбиения и после.
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        # gs - массив с неопределенностями для каждого разбиения.
        # Находим индекс соотвествующий лучшему разбиению.
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)

        # Что делает этот блок кода?
        # Получаем индекс left_el_id, после которого проходит разбиение.
        # Возвращаем неопределенность, и пороговое значение.
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        n_samples = x.shape[0]
        bins = np.bincount(y, minlength=self.num_class)

        if(self.max_depth is not None and self.max_depth == depth) or \
                n_samples < self.min_samples_split or \
                np.argmax(bins)/float(n_samples) >= self.sufficient_share:
            self.tree[node_id] = (self.LEAF_TYPE, np.argmax(bins))
            return
        else:
            n_features = x.shape[1]
            gs = np.zeros(n_features, dtype=float)
            thresholds = np.zeros(n_features, dtype=float)
            for i in self.get_feature_ids(n_features):
                gs[i], thresholds[i] = self.__find_threshold(x[:, i], y)
            feat_id_best = np.argmin(gs)
            threshold_best = thresholds[feat_id_best]

            # for counting correct importances
            classes = np.empty((1, bins.shape[0]))
            classes[0] = bins
            size = np.empty((1, 1))
            size[0, 0] = bins.sum()

            self.feature_importances_[feat_id_best] += \
                n_samples*self.G_supple_function(classes, size) - \
                gs[feat_id_best]
            self.feature_importances_[feat_id_best] /= n_samples

            x_l, x_r, y_l, y_r = self.__div_samples(x, y, feat_id_best,
                                                    threshold_best)

            if (x_l.shape[0] == 0 or x_r.shape[0] == 0):
                self.tree[node_id] = (self.LEAF_TYPE, np.argmax(bins))
                return

            self.tree[node_id] = (self.NON_LEAF_TYPE,
                                  feat_id_best, threshold_best)
            self.__fit_node(x_l, y_l, 2*node_id+1, depth+1, pred_f=-1)
            self.__fit_node(x_r, y_r, 2*node_id+2, depth+1, pred_f=-1)

    def fit(self, x, y):
        self.feature_importances_ = np.zeros(x.shape[1], dtype=float)
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [26]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [27]:
# %%pycodestyle
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data,
                                                    wine.target,
                                                    test_size=0.1,
                                                    stratify=wine.target)

## Проверка скорости работы на wine

In [28]:
%time clf.fit(X_train, y_train)

CPU times: user 4 ms, sys: 4 ms, total: 8 ms
Wall time: 3.92 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [29]:
%time my_clf.fit(X_train, y_train)

CPU times: user 28 ms, sys: 0 ns, total: 28 ms
Wall time: 27.9 ms


## Проверка качества работы на wine

In [30]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.882051282051282

In [31]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8888888888888888

## Подготовка данных Speed Dating Data 

In [9]:
# %%pycodestyle
df = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv',
                 encoding='cp1251')
df = df.iloc[:, :97]

df = df.drop(['id'], axis=1)
df = df.drop(['idg'], axis=1)
df = df.drop(['condtn'], axis=1)

df = df.drop(['round'], axis=1)
df = df.drop(['position', 'positin1'], axis=1)
df = df.drop(['order'], axis=1)
df = df.drop(['partner'], axis=1)
df = df.drop(['age_o', 'race_o', 'pf_o_att',
              'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o'],
             axis=1)
df = df.dropna(subset=['age'])


df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(19)

df = df.drop(['field'], axis=1)
df = df.drop(['undergra'], axis=1)

df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)

df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)

df.loc[:, 'tuition'] = \
    df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)

df.loc[:, 'tuition'] = df.tuition.fillna(-999)
df = df.dropna(subset=['imprelig', 'imprace'])
df = df.drop(['from', 'zipcode'], axis=1)
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
df = df.dropna(subset=['date'])
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(18)
df = df.drop(['career'], axis=1)
df = df.drop(['sports', 'tvsports', 'exercise', 'dining',
              'museums', 'art', 'hiking', 'gaming', 'clubbing',
              'reading', 'tv', 'theater', 'movies', 'concerts',
              'music', 'shopping', 'yoga'], axis=1)
df = df.drop(['expnum'], axis=1)
feat = ['iid', 'wave', 'attr1_1', 'sinc1_1', 'intel1_1',
        'fun1_1', 'amb1_1', 'shar1_1']
temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
idx = ((temp.wave < 6) | (temp.wave > 9)) & (temp.totalsum < 99)
idx = ((temp.wave >= 6) & (temp.wave <= 9))
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1',
                                        'intel1_1', 'fun1_1',
                                        'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
           'fun1_1', 'amb1_1', 'shar1_1']] = \
    (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1',
                'amb1_1', 'shar1_1']].T / df.loc[:, 'temp_totalsum'].T).T * 100

feat = ['iid', 'wave', 'attr2_1', 'sinc2_1',
        'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']
temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
idx = ((temp.wave < 6) | (temp.wave > 9)) & \
    (temp.totalsum < 90) & (temp.totalsum != 0)
temp.loc[idx, ]
idx = ((temp.wave >= 6) & (temp.wave <= 9))
temp.loc[idx, ]
df.loc[:, 'temp_totalsum'] = \
    df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
               'fun2_1', 'amb2_1', 'shar2_1']].sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']]\
    = (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                  'fun2_1', 'amb2_1', 'shar2_1']].T /
       df.loc[:, 'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)
for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
            'intel{}_1'.format(i), 'fun{}_1'.format(i),
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    df = df.drop(feat, axis=1)
df = df.drop(['wave'], axis=1)
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid']) \
    .drop(['gender'], axis=1) \
    .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid']) \
    .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1) \
    .dropna()
df_female.columns = df_female.columns + '_f'
df_pair = df_male.join(df_female.set_index('iid_f'), on='pid', how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)
SD_data = df_pair.iloc[:, 1:].values
SD_target = df_pair.iloc[:, 0].values

In [38]:
# %%pycodestyle
X_train, X_test, y_train, y_test = \
    train_test_split(SD_data, SD_target, test_size=0.1, stratify=SD_target)

In [39]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы на Speed Dating Data 

In [40]:
%time clf.fit(X_train, y_train)

CPU times: user 92 ms, sys: 0 ns, total: 92 ms
Wall time: 92.9 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [41]:
%time my_clf.fit(X_train, y_train)

CPU times: user 1.01 s, sys: 0 ns, total: 1.01 s
Wall time: 1.01 s


## Проверка качества работы на Speed Dating Data

In [42]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5475681789619203

In [43]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.5403472931562819

## Задание 3

In [16]:
from sklearn.pipeline import Pipeline

In [17]:
# %%pycodestyle
estimators = [('clf', DecisionTreeClassifier())]
pipe = Pipeline(estimators)

In [18]:
# %%pycodestyle
pipe.fit(X_train, y_train)
f1_score(y_pred=pipe.predict(X_test), y_true=y_test, average='macro')

0.5624589805294247

## Задание 4

#### DecisionTreeClassifier

In [19]:
# %%pycodestyle
clf2 = DecisionTreeClassifier()
clf2.fit(X_train, y_train)
print(pd.Series(index=df_pair.columns[1:], data=clf2.feature_importances_).
      sort_values(ascending=False).head(10))

int_corr      0.077964
age           0.038029
sinc1_1       0.034003
age_f         0.033349
attr1_1_f     0.029455
amb3_1_f      0.026034
exphappy      0.025516
intel1_1_f    0.024553
amb1_1_f      0.022482
income        0.021632
dtype: float64


#### myDecisionTreeClassifier

In [20]:
# %%pycodestyle
myclf2 = MyDecisionTreeClassifier()
myclf2.fit(X_train, y_train)
print(pd.Series(index=df_pair.columns[1:], data=myclf2.feature_importances_).
      sort_values(ascending=False).head(10))

intel1_1_f    0.525909
age_f         0.500000
attr1_1_f     0.468750
age           0.309202
samerace      0.303043
sinc1_1       0.281250
career_c      0.261433
intel3_1_f    0.261224
date_f        0.250780
amb2_1_f      0.241343
dtype: float64


## Задание 5

In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [22]:
# %%pycodestyle
grid_params = {'criterion': ['gini', 'entropy'],
               'max_depth': range(2, 21, 2),
               'min_samples_split': range(2, 21, 2),
               }
n_iter = 30

In [23]:
# %%pycodestyle
rscv1 = GridSearchCV(RandomForestClassifier(), grid_params)
rscv1.fit(X_train, y_train)
print(rscv1.best_params_)

{'max_depth': 16, 'criterion': 'gini', 'min_samples_split': 12}
